# 📅 `Automate the Daily Excel Work to Prepare WIP Report` :  
> My objective here is to **completely Automate the manual Excel Work** to prepare the WIP(Work In Progress) Data and It's Status in **PPMC (Pre-Policy Medical Check-Up)** so that we can reduce the **45 Minutes of manual work to 3 Minutes** on daily basis for 25 Account Managers.


## 🔰 `Introduction` :
>**PPMC :** In the context of insurance, PPMC stands for Pre-Policy Medical Check-Up. This is a set of medical examinations and tests that a prospective policyholder must undergo before an insurance company approves their policy application. Here's a brief overview:

>**What is a Pre-Policy Medical Check-Up (PPMC) ?** <br>
A PPMC consists of various medical tests to assess the applicant's health status. These tests help the insurer determine the applicant's medical fitness and identify any pre-existing conditions. The results of these tests can influence the premium rates and coverage terms of the policy.

>**Importance of PPMC:** <br>
>1. Assessing Health Status: It provides a baseline health assessment for the policyholder.
>2. Identifying Pre-Existing Conditions: Helps insurers understand if there are any existing medical conditions that need to be considered.
>3. Determining Premiums: The results can affect the premium rates, ensuring they are appropriate for the individual's health status.
>4. Smooth Claim Settlement: The medical reports from the PPMC can be crucial during the claim settlement process.

>**Who Needs to Undergo PPMC ?** <br>
Typically, policyholders above a certain age (often 40 or 45 years) are required to undergo a PPMC. However, this can vary depending on the insurance company and the type of policy.


## 📊 `The Source Data` :
> Account manageers extract csv file from CRM in evening to prepare the WIP Report. This file have **185 Columns** and usually **lakhs of rows** based on their insurer volumes. They use the following columns to prepare the next day appointment tracker :
>- **CorporateName :** Insurer Name
>- **PatientName :** Name of Insured who will go under the medicals.
>- **ApplicationId :** Insured Application Number
>- **AppointmentStatus :** Current starus of Appointment
>- **LastCallStatus :** Final Call Status
>- **NumberofAttempts :** No of Call Attempts
>- **DND :** Particular Case has been marked DND (Do Not Disturb) or not.


## ⭐ `Getting started` :
>We will first build the logic and will explain and make the program. And after the we will make a function to repeat this daily task, make it simple for the account manager who is not well verse with python and save the time.

#### `Import Library` 

In [136]:
import pandas as pd
from datetime import datetime

#### `Load the raw data with selected columns into our Pandas Data Frame df`

In [139]:
df_raw = pd.read_csv("mis.csv", usecols=["RequestDate", "PatientName", "ApplicationId", "AppointmentStatus", "LastCallStatus", 
                                         "NumberofAttempts", "DND"])

In [141]:
# Check the data information 
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7111 entries, 0 to 7110
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   RequestDate        7111 non-null   object 
 1   PatientName        7111 non-null   object 
 2   ApplicationId      7111 non-null   object 
 3   AppointmentStatus  7111 non-null   object 
 4   LastCallStatus     4050 non-null   object 
 5   NumberofAttempts   5826 non-null   float64
 6   DND                7111 non-null   object 
dtypes: float64(1), object(6)
memory usage: 389.0+ KB


#### `Convert the RequestDate column to datetime format and copy to new dataframe df` 

In [144]:
df = df_raw.copy()
df["RequestDate"] = pd.to_datetime(df["RequestDate"], format="%d/%m/%Y", errors="coerce")

In [146]:
# Check df info now
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7111 entries, 0 to 7110
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   RequestDate        7111 non-null   datetime64[ns]
 1   PatientName        7111 non-null   object        
 2   ApplicationId      7111 non-null   object        
 3   AppointmentStatus  7111 non-null   object        
 4   LastCallStatus     4050 non-null   object        
 5   NumberofAttempts   5826 non-null   float64       
 6   DND                7111 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 389.0+ KB


#### `Remove previous financial year's data`

In [149]:
financial_year_end = datetime(2024, 3, 31)
df = df[df["RequestDate"] > financial_year_end]

In [151]:
# Check the Data now
df.head()

,RequestDate,PatientName,ApplicationId,AppointmentStatus,LastCallStatus,NumberofAttempts,DND
0,2024-10-18,Rohit Makwana,GMDBA10030_1000000030_GL0301,Appointment Attended,NaN,56.0,No
1,2024-10-21,Divya Bhalla,GMDMU11047_1000000017_GL0301,Appointment Attended,NaN,50.0,No
2,2024-11-07,Sriteja Kolluri,GMDBA09838_1000000046_GL0301,Appointment Attended,Appointment Request Received,12.0,No
3,2024-11-09,DINESH KUMAR GIRDHAR,C266196538,Appointment Attended,NaN,3.0,No
4,2024-11-09,NIDHI CHATURVEDI,GMDDE09747_1000000020_GL0301,Appointment Attended,NaN,2.0,No


#### `Remove completed cases`

In [156]:
completed_statuses = ["QC Approved", "QC APPROVED", "Reports Uploaded", "Appointment Attended", "QC Rejected", "Sent For Interpretation"]
df = df[~df["AppointmentStatus"].isin(completed_statuses)]

In [158]:
# Check the Data now
df.head()

,RequestDate,PatientName,ApplicationId,AppointmentStatus,LastCallStatus,NumberofAttempts,DND
25,2024-10-15,AMIR ALI,GMDDE09747_1000000004_GL0301,Appointment Confirmed,Non Contactable,14.0,No
26,2024-10-21,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,13.0,No
27,2024-10-25,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,1.0,No
28,2024-11-06,Sabyasachi Das,GMDMU08621_1000000006_GL0301,Appointment Confirmed,NaN,2.0,No
29,2024-11-08,Prakash Chandra Vatsa,GMDMU09654_1000000004_GL0301,Appointment Confirmed,Appointment Request Received,4.0,No


## `Categorize rows based on various conditions`

#### `Add a new Column status and initialize it.`

In [162]:
# Initialize the Status column
df["Status"] = None  

In [164]:
# Check the Data now
df.head()

,RequestDate,PatientName,ApplicationId,AppointmentStatus,LastCallStatus,NumberofAttempts,DND,Status
25,2024-10-15,AMIR ALI,GMDDE09747_1000000004_GL0301,Appointment Confirmed,Non Contactable,14.0,No,None
26,2024-10-21,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,13.0,No,None
27,2024-10-25,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,1.0,No,None
28,2024-11-06,Sabyasachi Das,GMDMU08621_1000000006_GL0301,Appointment Confirmed,NaN,2.0,No,None
29,2024-11-08,Prakash Chandra Vatsa,GMDMU09654_1000000004_GL0301,Appointment Confirmed,Appointment Request Received,4.0,No,None


#### `Data with Maximum Call Attempts`

In [167]:
df.loc[df["NumberofAttempts"] > 30, "Status"] = "Max Attempts"

In [169]:
# Check the Data now
df[df["Status"]=="Max Attempts"]

,RequestDate,PatientName,ApplicationId,AppointmentStatus,LastCallStatus,NumberofAttempts,DND,Status
130,2024-04-03,Nishith Mehta,GMDMU08335_1000000077_GL0301,Appointment Unattended,Non Contactable,57.0,No,Max Attempts
132,2024-04-06,Abhishek Bharti,GMDDE09854_1000000707_GL0301,Appointment Unattended,Non Contactable,97.0,No,Max Attempts
134,2024-04-08,Asif Sajjad,GMDBA09724_1000000060_GL0301,Appointment Unattended,Non Contactable,39.0,No,Max Attempts
135,2024-04-15,Deepesh Moni,GMDMU09095_1000000014_GL0301,Appointment Unattended,Non Contactable,58.0,No,Max Attempts
136,2024-04-23,Ankit Agarwal,GMDBA09852_1000000006_GL0301,Appointment Unattended,NaN,37.0,Yes,Max Attempts
...,...,...,...,...,...,...,...,...
7066,2024-10-29,Sathish Srinivasan,GMDMU11047_1000000019_GL0301,WIP,Non Contactable,46.0,No,Max Attempts
7068,2024-10-30,Paras Nayyar,GMDHY09734_1000000064_GL0301,WIP,Non Contactable,38.0,No,Max Attempts
7069,2024-10-30,Sumit Gupta,GMDHY09734_1000000065_GL0301,WIP,Non Contactable,41.0,No,Max Attempts
7070,2024-10-30,Vishal Agrawal,GMDHY09734_1000000062_GL0301,WIP,Reminder,38.0,No,Max Attempts


#### `DND Data`

In [172]:
df.loc[(df["Status"].isna()) & (df["DND"] == "Yes"), "Status"] = "DND"

#### `Assign Appointment Status Cases to the Status Column`

In [175]:
appointment_status_cases = ["Cancelled", "Cancelled by insurer", "Appointment Confirmed", "Order sent to partner"]
df.loc[(df["AppointmentStatus"].isin(appointment_status_cases)) & (df["Status"].isna()), "Status"] = df["AppointmentStatus"]

In [177]:
# Check the data now
df

,RequestDate,PatientName,ApplicationId,AppointmentStatus,LastCallStatus,NumberofAttempts,DND,Status
25,2024-10-15,AMIR ALI,GMDDE09747_1000000004_GL0301,Appointment Confirmed,Non Contactable,14.0,No,Appointment Confirmed
26,2024-10-21,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,13.0,No,Appointment Confirmed
27,2024-10-25,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,1.0,No,Appointment Confirmed
28,2024-11-06,Sabyasachi Das,GMDMU08621_1000000006_GL0301,Appointment Confirmed,NaN,2.0,No,Appointment Confirmed
29,2024-11-08,Prakash Chandra Vatsa,GMDMU09654_1000000004_GL0301,Appointment Confirmed,Appointment Request Received,4.0,No,Appointment Confirmed
...,...,...,...,...,...,...,...,...
7106,2024-11-15,Giridhar G M,GMDMU09088_1000001173_GL0301,WIP,NaN,NaN,Yes,DND
7107,2024-11-15,Gaurav Malik,GMDBA10022_1000000007_GL0301,WIP,Non Contactable,3.0,No,None
7108,2024-11-15,Abhishek Kumar Jha\t,U225774357,WIP,Insurer Actionable,1.0,No,None
7109,2024-11-15,Pravin Shivaji Hadawale,GMDMU09555_1000000031_GL0301,WIP,Non Contactable,3.0,No,None


#### `Assign Last Call Status to the remaining Status Rows`

In [180]:
# Last Call Status
df.loc[df["Status"].isna(), "Status"] = df["LastCallStatus"]

In [182]:
# Check Data Info
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1577 entries, 25 to 7110
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   RequestDate        1577 non-null   datetime64[ns]
 1   PatientName        1577 non-null   object        
 2   ApplicationId      1577 non-null   object        
 3   AppointmentStatus  1577 non-null   object        
 4   LastCallStatus     1288 non-null   object        
 5   NumberofAttempts   1384 non-null   float64       
 6   DND                1577 non-null   object        
 7   Status             1543 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 143.2+ KB


In [184]:
# Check remaning Null count on status column as LastCallStatus null values has assigned to status column
df["Status"].isna().sum()

34

#### `Fill remaining null values in Status with "Non Contactable"`

In [187]:
df["Status"] = df["Status"].fillna("Non Contactable")

In [189]:
# Now Check Null count on status column
df["Status"].isna().sum()

0

#### `Categorize "Working On It" cases`

In [192]:
working_on_it_statuses = ["Appointment Request Received", "Direct Medical", "Location Constraint", "Medical Done Report Awaited", "Reminder"]
df.loc[df["Status"].isin(working_on_it_statuses), "Status"] = "Working On It"

In [194]:
# Check the data now
df

,RequestDate,PatientName,ApplicationId,AppointmentStatus,LastCallStatus,NumberofAttempts,DND,Status
25,2024-10-15,AMIR ALI,GMDDE09747_1000000004_GL0301,Appointment Confirmed,Non Contactable,14.0,No,Appointment Confirmed
26,2024-10-21,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,13.0,No,Appointment Confirmed
27,2024-10-25,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,1.0,No,Appointment Confirmed
28,2024-11-06,Sabyasachi Das,GMDMU08621_1000000006_GL0301,Appointment Confirmed,NaN,2.0,No,Appointment Confirmed
29,2024-11-08,Prakash Chandra Vatsa,GMDMU09654_1000000004_GL0301,Appointment Confirmed,Appointment Request Received,4.0,No,Appointment Confirmed
...,...,...,...,...,...,...,...,...
7106,2024-11-15,Giridhar G M,GMDMU09088_1000001173_GL0301,WIP,NaN,NaN,Yes,DND
7107,2024-11-15,Gaurav Malik,GMDBA10022_1000000007_GL0301,WIP,Non Contactable,3.0,No,Non Contactable
7108,2024-11-15,Abhishek Kumar Jha\t,U225774357,WIP,Insurer Actionable,1.0,No,Insurer Actionable
7109,2024-11-15,Pravin Shivaji Hadawale,GMDMU09555_1000000031_GL0301,WIP,Non Contactable,3.0,No,Non Contactable


#### `Add a new Column and Categorize Type as "Workable" or "Non_Workable"`

In [197]:
workable_statuses = ["Appointment Confirmed", "Callback", "Non Contactable", "Order sent to partner", "Working On It"]
df["Type"] = "Non_Workable"  # Default to Non_Workable
df.loc[df["Status"].isin(workable_statuses), "Type"] = "Workable"

In [199]:
# Check data now
df

,RequestDate,PatientName,ApplicationId,AppointmentStatus,LastCallStatus,NumberofAttempts,DND,Status,Type
25,2024-10-15,AMIR ALI,GMDDE09747_1000000004_GL0301,Appointment Confirmed,Non Contactable,14.0,No,Appointment Confirmed,Workable
26,2024-10-21,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,13.0,No,Appointment Confirmed,Workable
27,2024-10-25,Anand Kamath,GMDMU11047_1000000013_GL0301,Appointment Confirmed,Appointment Request Received,1.0,No,Appointment Confirmed,Workable
28,2024-11-06,Sabyasachi Das,GMDMU08621_1000000006_GL0301,Appointment Confirmed,NaN,2.0,No,Appointment Confirmed,Workable
29,2024-11-08,Prakash Chandra Vatsa,GMDMU09654_1000000004_GL0301,Appointment Confirmed,Appointment Request Received,4.0,No,Appointment Confirmed,Workable
...,...,...,...,...,...,...,...,...,...
7106,2024-11-15,Giridhar G M,GMDMU09088_1000001173_GL0301,WIP,NaN,NaN,Yes,DND,Non_Workable
7107,2024-11-15,Gaurav Malik,GMDBA10022_1000000007_GL0301,WIP,Non Contactable,3.0,No,Non Contactable,Workable
7108,2024-11-15,Abhishek Kumar Jha\t,U225774357,WIP,Insurer Actionable,1.0,No,Insurer Actionable,Non_Workable
7109,2024-11-15,Pravin Shivaji Hadawale,GMDMU09555_1000000031_GL0301,WIP,Non Contactable,3.0,No,Non Contactable,Workable


In [201]:
# Format RequestDate back to string format
df["RequestDate"] = df["RequestDate"].dt.strftime("%d/%m/%Y")

In [203]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1577 entries, 25 to 7110
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   RequestDate        1577 non-null   object 
 1   PatientName        1577 non-null   object 
 2   ApplicationId      1577 non-null   object 
 3   AppointmentStatus  1577 non-null   object 
 4   LastCallStatus     1288 non-null   object 
 5   NumberofAttempts   1384 non-null   float64
 6   DND                1577 non-null   object 
 7   Status             1577 non-null   object 
 8   Type               1577 non-null   object 
dtypes: float64(1), object(8)
memory usage: 155.5+ KB


#### `Generate a summary report of Status counts`

In [208]:
df_status = df["Status"].value_counts()

In [220]:
df_status

Status
Cancelled by insurer     484
Max Attempts             323
Insurer Actionable       220
DND                      197
Cancelled                118
Non Contactable          108
Working On It             76
Order sent to partner     19
Callback                  16
Appointment Confirmed     12
Duplicate                  4
Name: count, dtype: int64

### `Generate the Workable and Non-Workable Data`

In [223]:
grouped_report = ( df.groupby(["Type", "Status"]).size().reset_index(name="Count"))

In [225]:
# Check the report
grouped_report

,Type,Status,Count
0,Non_Workable,Cancelled,118
1,Non_Workable,Cancelled by insurer,484
2,Non_Workable,DND,197
3,Non_Workable,Duplicate,4
4,Non_Workable,Insurer Actionable,220
5,Non_Workable,Max Attempts,323
6,Workable,Appointment Confirmed,12
7,Workable,Callback,16
8,Workable,Non Contactable,108
9,Workable,Order sent to partner,19


### `Export data to the different sheets in same Excel Workbook`

In [228]:
with pd.ExcelWriter("WIP_Report.xlsx") as writer:
    grouped_report.to_excel(writer, sheet_name="Status_Report", index=False)
    df_raw.to_excel(writer, sheet_name="Raw_Data", index=False)
    df.to_excel(writer, sheet_name="Cleaned_Data", index=False)